In [39]:
# import nest_asyncio
# nest_asyncio.apply()
import os
import django
import inspect

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
    
from books.models import Book
from books import utils

from asgiref.sync import sync_to_async
from django.shortcuts import get_object_or_404
from django.http import HttpRequest

In [57]:
from django.db import connection
connection.settings_dict['PASSWORD'] = '5207'
dir(HttpRequest().POST)
'd467227cab2b40a8' in request.values()

True

In [63]:
b = Book.objects.raw("select * from books_book as t where title=$$%s$$" % '11ef11b1d80aa8e0')
# from django.core.serializers import serialize
# serialize('json', b)
list(b)[0].get_absolute_url()


NoReverseMatch: Reverse for 'book-detail' not found. 'book-detail' is not a valid view function or pattern name.

In [36]:
lookup_fields = ['title', 'serial_number']
request = {'title':'d467227cab2b40a8','serial_number':"sdf"}
kwargs = {i:request[i] for i in lookup_fields if i in request}
kwargs


{'title': 'd467227cab2b40a8', 'serial_number': 'sdf'}

In [38]:
def get_sql_lookups(lookup_dict, lookup_fields):
    q = [f"table.{i}=$${j}$$" for (i, j) in lookup_dict.items()
         if i in lookup_fields]
    return ''.join(q)
get_sql_lookups(request)

'table.title=$$d467227cab2b40a8$$table.serial_number=$$sdf$$'

In [11]:
from django.contrib.admin.options import modelform_factory

f = modelform_factory(Book, fields=['serial_number'])
Book._meta.verbose_name
f['serial_number']

TypeError: 'ModelFormMetaclass' object is not subscriptable

In [95]:
from faker import Faker
from time import time
import time as T
import random
import string

import hashlib


def generate_random_string(length):
    letters = string.ascii_letters
    random_string = ''.join(random.sample(string.ascii_letters,length))
    return random_string

f = Faker()
t1 = time()
for i in range(1000000):
    # ''.join(random.sample(string.ascii_letters,10))
    # f.uuid4()
    data = f"Hello, World!{T.time()}".encode()
    x.update(data)
    x.hexdigest()
    f.text(10)
    
t2 = time()
t2-t1

7.442434072494507

In [29]:
generate_random_string(12)

'dxlbGLpcnEJM'

In [26]:
''.join(random.sample(string.ascii_letters,10))

'KGZDIEQgyi'

In [82]:
vars(Book._meta)
Book._meta.get_fields()

(<django.db.models.fields.BigAutoField: id>,
 <django.db.models.fields.CharField: title>,
 <django.db.models.fields.DateField: publication_date>,
 <django.db.models.fields.FloatField: price>,
 <django.db.models.fields.CharField: serial_number>,
 <django.db.models.fields.TextField: description>)

In [77]:
inspect.getmembers(Book)

[('DoesNotExist', books.models.Book.DoesNotExist),
 ('MultipleObjectsReturned', books.models.Book.MultipleObjectsReturned),
 ('__class__', django.db.models.base.ModelBase),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  mappingproxy({'__module__': 'books.models',
                '__str__': <function books.models.Book.__str__(self)>,
                '__doc__': 'Book(id, title, publication_date, price, serial_number, description)',
                '_meta': <Options for Book>,
                'DoesNotExist': books.models.Book.DoesNotExist,
                'MultipleObjectsReturned': books.models.Book.MultipleObjectsReturned,
                'title': <django.db.models.query_utils.DeferredAttribute at 0x7f373e1cb0d0>,
                'publication_date': <django.db.models.query_utils.DeferredAttribute at 0x7f373e1cb130>,
                'get_next_by_publication_date': functools.partialmethod(<function Model._get_next_or_previous_by_FIELD at 0x7f3754329b40>